In [21]:
import pandas as pd
import psycopg2

In [22]:
def extract_transform():
    df = pd.read_csv('../data/Udemy.csv')
    df = df.round(decimals = 1)
    df[['course_id', 'price', 'num_subscribers', 'num_reviews', 'num_lectures']] = df[['course_id', 'price', 'num_subscribers', 'num_reviews', 'num_lectures']].astype('Int64')
    df['published_timestamp'] = pd.to_datetime(df['published_timestamp'])
    df = df.dropna()
    return df

def load_to_database(data):
    try:
        conn = psycopg2.connect(
            host = '127.0.0.1',
            database = 'Udemy',
            user = 'postgres',
            password = 'postgres',
            port = 5432
        )
        cur = conn.cursor()


        sql = "INSERT INTO courses (course_id,course_title,url,price,num_subscribers,num_reviews,num_lectures,level,Rating,content_duration,published_timestamp,subject) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        cur.executemany(sql, data.values.tolist())
        conn.commit()

        print('Data Inserted Successfully')

    except psycopg2.OperationalError as e:
        print('Failed to connect:', e)

    finally:
        cur.close()
        if conn:
            conn.close()

def etl():
    df = extract_transform()
    load_to_database(df)

etl()

UniqueViolation: duplicate key value violates unique constraint "courses_pkey"
DETAIL:  Key (course_id)=(348116) already exists.


In [23]:
# import wget
# wget.download(url="https://davidmegginson.github.io/ourairports-data/airports.csv", out="../data/airports/")
# wget.download(url="https://davidmegginson.github.io/ourairports-data/airport-frequencies.csv", out="../data/airports/")
# wget.download(url="https://davidmegginson.github.io/ourairports-data/runways.csv", out="../data/airports/")
# wget.download(url="https://davidmegginson.github.io/ourairports-data/navaids.csv", out="../data/airports/")
# wget.download(url="https://davidmegginson.github.io/ourairports-data/countries.csv", out="../data/airports/")
# wget.download(url="https://davidmegginson.github.io/ourairports-data/regions.csv", out="../data/airports/")

In [34]:
import os
path = "../data/airports"

def extract_data_sources():
    if not os.path.exists(path):
        os.makedirs(path)

    for i in os.listdir(path):
        os.remove(path + '/' + i)

    os.system('wget -P ../data/airports/ -i ../data/airports/airport.txt')

In [25]:
import dotenv

In [26]:
dotenv.load_dotenv("../.env")

True

In [28]:
from sqlalchemy import create_engine
import pandas as pd
import snowflake.connector

In [29]:
def move_to_snowflake():
    snowflake_account   = "drb29258.us-east-1"
    snowflake_user      = "rojacobeus"
    snowflake_password  = os.getenv("SNOWFLAKE_PASSWORD")
    snowflake_database  = "STAGING"
    snowflake_schema    = "AIR_STAGING"
    snowflake_warehouse = "COMPUTE_WH"

    # snowflake connection
    conn = snowflake.connector.connect(
        user      = snowflake_user,
        password  = snowflake_password,
        account   = snowflake_account,
        warehosue = snowflake_warehouse,
        database  = snowflake_database,
        schema    = snowflake_schema
    )

    engine = create_engine(f'snowflake://{snowflake_user}:{snowflake_password}@{snowflake_account}/{snowflake_database}/{snowflake_schema}')
    
    airports = pd.read_csv('../data/airports/airports.csv')
    airports.to_sql(name="airports", con=engine, if_exists='replace', index=False, chunksize=16000) 

    airport_frequencies = pd.read_csv('../data/airports/airport-frequencies.csv')
    airport_frequencies.to_sql("airport_frequencies", engine, if_exists='replace', index=False, chunksize=16000)

    navaids = pd.read_csv('../data/airports/navaids.csv')
    navaids.to_sql("navaids", engine, if_exists='replace', index=False, chunksize=16000)

    countries = pd.read_csv('../data/airports/countries.csv')
    countries.to_sql("countries", engine, if_exists='replace', index=False, chunksize=16000)

    regions = pd.read_csv('../data/airports/regions.csv')
    regions.to_sql("regions", engine, if_exists='replace', index=False, chunksize=16000)

    runways = pd.read_csv('../data/airports/runways.csv')
    runways.to_sql("runways", engine, if_exists='replace', index=False, chunksize=16000)

    print('successfully moved to snowflake staging air table')
    conn.close()

In [30]:
snowflake_account   = "drb29258.us-east-1"
snowflake_user      = "rojacobeus"
snowflake_password  = os.getenv("SNOWFLAKE_PASSWORD")
snowflake_database  = "STAGING"
snowflake_schema    = "AIR_STAGING"
snowflake_warehouse = "COMPUTE_WH"


In [31]:
engine = create_engine(f'snowflake://{snowflake_user}:{snowflake_password}@{snowflake_account}/{snowflake_database}/{snowflake_schema}')

In [19]:
airports = pd.read_csv('../data/airports/airports.csv')
airports.to_sql(name="airports", con=engine, if_exists='replace', index=False, chunksize=16000) 

AttributeError: 'NoneType' object has no attribute 'get'

In [15]:
move_to_snowflake()

AttributeError: 'NoneType' object has no attribute 'get'